<a href="https://colab.research.google.com/github/adeepH/CASE-2021-Task-1/blob/main/Notebooks/soft_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
sen=[]
l=[]
with open('/content/drive/MyDrive/en-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    l.append(x['label'])
    sen.append(x['sentence'].strip())

In [ ]:
ssen=[]
sl=[]
with open('/content/drive/MyDrive/es-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    sl.append(x['label'])
    ssen.append(x['sentence'].strip())
psen=[]
pl=[]
with open('/content/drive/MyDrive/pr-train.json') as f:
  c=0
  for i in f.readlines():
    x=json.loads(i)
    pl.append(x['label'])
    psen.append(x['sentence'].strip())

In [ ]:
x=sen
x.extend(ssen)
x.extend(psen)
y=l
y.extend(sl)
y.extend(pl)

In [ ]:
from sklearn.model_selection import train_test_split
x,x1,y,y1=train_test_split(x,y,test_size=0.2)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 22.6MB/s 
     |████████████████████████████████| 901kB 23.8MB/s 


In [ ]:
from transformers import RobertaTokenizer
from transformers import TFRobertaModel

In [ ]:
r_tokenizer=RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
r_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling1D,Dropout,LSTM
from tensorflow.keras.models import Model
def create_roberta_model():
  inp1=Input(shape=(128,),dtype='int32')
  inp2=Input(shape=(128,),dtype='int32')
  emb=r_model(inp1,attention_mask=inp2)[0]
  l=CuDNNLSTM(128,return_sequences=True)(emb)
  l=GlobalAveragePooling1D()(l)
  l=Dense(50,activation='relu')(l)
  l=Dropout(0.2)(l)
  l=Dense(1,activation='sigmoid')(l)
  r_m=Model(inputs=[inp1,inp2],outputs=l)
  return r_m
r_m=create_roberta_model()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
d1=r_tokenizer(x1,padding=True,truncation=True,return_token_type_ids=False,max_length=128)

In [ ]:
import numpy as np
q1=np.array(d1['input_ids'])
q2=np.array(d1['attention_mask'])

In [ ]:
import keras
r_m.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
r_m.load_weights('/content/drive/MyDrive/roberta base model')

In [ ]:
pred=r_m.predict([q1,q2])
p=[0 if i[0]<=0.5 else 1 for i in pred]

In [ ]:
#Classification report on Roberta Model on Validation Set
from sklearn.metrics import classification_report
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.910     0.938     0.924      4387
           1      0.670     0.578     0.621       963

    accuracy                          0.873      5350
   macro avg      0.790     0.758     0.772      5350
weighted avg      0.867     0.873     0.869      5350



In [ ]:
from transformers import BertTokenizer
from transformers import TFBertModel

In [ ]:
b_t=BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
b_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_bert_model():
  inp1=Input(shape=(128,),dtype='int32')
  inp2=Input(shape=(128,),dtype='int32')
  emb=b_model(inp1,attention_mask=inp2)[0]
  l=CuDNNLSTM(128,return_sequences=True)(emb)
  l=GlobalAveragePooling1D()(l)
  l=Dense(50,activation='relu')(l)
  l=Dropout(0.2)(l)
  l=Dense(1,activation='sigmoid')(l)
  b_m=Model(inputs=[inp1,inp2],outputs=l)
  return b_m
b_m=create_bert_model()

In [ ]:
d1=b_t(x1,padding=True,truncation=True,return_token_type_ids=False,max_length=128)

In [ ]:
b_m.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
q1=np.array(d1['input_ids'])
q2=np.array(d1['attention_mask'])

In [ ]:
b_m.load_weights('/content/drive/MyDrive/bert multi cased model')

In [ ]:
bert_pred=b_m.predict([q1,q2])
p=[0 if i[0]<=0.5 else 1 for i in bert_pred]

In [ ]:
#Classification report on Bert Model on Validation Set
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.928     0.917     0.922      4387
           1      0.641     0.675     0.658       963

    accuracy                          0.873      5350
   macro avg      0.784     0.796     0.790      5350
weighted avg      0.876     0.873     0.875      5350



In [ ]:
#to free up space
del(r_tokenizer)

In [ ]:
del(b_t)

In [ ]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertModel

In [ ]:
d_t=DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [ ]:
d_model = TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
def create_distilbert_model():
  inp1=Input(shape=(128,),dtype='int32')
  inp2=Input(shape=(128,),dtype='int32')
  emb=d_model(inp1,attention_mask=inp2)[0]
  l=CuDNNLSTM(128,return_sequences=True)(emb)
  l=GlobalAveragePooling1D()(l)
  l=Dense(50,activation='relu')(l)
  l=Dropout(0.2)(l)
  l=Dense(1,activation='sigmoid')(l)
  d_m=Model(inputs=[inp1,inp2],outputs=l)
  return d_m
d_m=create_distilbert_model()

In [ ]:
d1=d_t(x1,padding=True,truncation=True,return_token_type_ids=False,max_length=128)

In [ ]:
d_m.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-5),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
q1=np.array(d1['input_ids'])
q2=np.array(d1['attention_mask'])

In [ ]:
d_m.load_weights('/content/drive/MyDrive/ditilbert multi model')

In [ ]:
distilbert_pred=d_m.predict([q1,q2])
p=[0 if i[0]<=0.5 else 1 for i in distilbert_pred]

In [ ]:
#Classification report on DistilBert on Validation Set
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.924     0.947     0.936      4387
           1      0.729     0.646     0.685       963

    accuracy                          0.893      5350
   macro avg      0.827     0.797     0.810      5350
weighted avg      0.889     0.893     0.891      5350



In [ ]:
def soft_voting(a,b,c):
  a=a.flatten()
  b=b.flatten()
  c=c.flatten()
  v=(a+b+c)/3
  return v

In [ ]:
#pred indicates roberta predictions
#bert_pred indicates bert predictions
#distilbert_pred indicates distilbert predictions
res=soft_voting(pred,bert_pred,distilbert_pred)

In [ ]:
res

array([0.7729578 , 0.00482321, 0.81779546, ..., 0.04575844, 0.02386683,
       0.53771186], dtype=float32)

In [ ]:
bert_pred

array([[0.7387022 ],
       [0.00279794],
       [0.83110535],
       ...,
       [0.0532604 ],
       [0.05416551],
       [0.49774358]], dtype=float32)

In [ ]:
distilbert_pred

array([[0.89332235],
       [0.00175474],
       [0.8816272 ],
       ...,
       [0.01241302],
       [0.00593559],
       [0.6974833 ]], dtype=float32)

In [ ]:
pred

array([[0.6868487 ],
       [0.00991695],
       [0.7406536 ],
       ...,
       [0.07160189],
       [0.01149939],
       [0.41790876]], dtype=float32)

In [ ]:
p=[0 if i<=0.5 else 1 for i in res]

In [ ]:
#Classification report on Soft Voting on Validation Set
print(classification_report(y1,p,digits=3))

              precision    recall  f1-score   support

           0      0.937     0.939     0.938      4387
           1      0.720     0.713     0.717       963

    accuracy                          0.899      5350
   macro avg      0.829     0.826     0.827      5350
weighted avg      0.898     0.899     0.898      5350

